TODO:
##### Speech concatenate continuous chunks of audio?

In [ ]:
# !pip install noisereduce

In [1]:
!python stt_t2t_no_queue.py \

: 

In [ ]:
import pyaudio
import numpy as np
from queue import Queue
import atexit
import sounddevice as sd
import noisereduce as nr
from collections import Counter
import requests


import asyncio
import concurrent.futures
import threading
import time
import multiprocessing
from multiprocessing import Pool

import time



In [ ]:
# import multiprocessing as mp

# list1 = []

# def foo(list1, q):
#     q.put('hello')
#     list1.append('hello')

# if __name__ == '__main__':
#     # mp.set_start_method('spawn')
#     q = mp.Queue()
#     p = mp.Process(target=foo, args=(list1,q))
#     p.start()
#     # print(q.get(False))
#     p.join()
#     print(q.get_nowait())

    
#     # time.sleep(5)
#     # print(list1)

In [ ]:
# Constants for audio stream configuration
FORMAT = pyaudio.paInt16  # Sample format (16-bit)
CHANNELS = 1             # Number of audio channels (1 for mono, 2 for stereo)
# RATE = 44100             # Sample rate (samples per second)
# CHUNK_SIZE = 44100        # Number of audio frames per buffer
CHUNK_SIZE = 16000        # Number of audio frames per buffer
SAMPLING_RATE = 16000     # Sample rate (samples per second)

DETECT_THRESHOLD = 1000   # Threshold to detect a clap
DETECT_DURATION = 0.1    # Time in seconds to detect a clap
RECORD_DURATION = 3      # Time in seconds to record audio after clap is detected
MAX_RECORD_DURATION = 20 # Maximum time in seconds to record audio


In [ ]:
import whisper

model = whisper.load_model("small")
# model = whisper.load_model("medium")


In [ ]:
# def inference(audio):
async def inference(audio):
    if type(audio) == str:
        audio = whisper.load_audio(audio)
    print(type(audio), audio.dtype, audio.shape)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    # _, probs = model.detect_language(mel)
    
    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)
    
    print(result.text)
    return result.text


In [ ]:
def text_filter(text):
    
    char_count = Counter(text)
    # print(char_count)
    n_chars = len(char_count.keys())
    print("n_chars = ", n_chars)
    flag_n_chars_too_little = n_chars < 4
    # flag_any_char_too_many = np.all(np.array(list(char_count.values())) > 20)

    # return flag_n_chars_too_little and flag_any_char_too_many
    return flag_n_chars_too_little 

In [ ]:
from pynput import keyboard

def on_press(key):
    if key == keyboard.Key.esc:
        # Stop listener
        return False

def on_release(key):
    pass

# ...or, in a non-blocking fashion:
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()

In [ ]:
async def stt_func(audio_data):
    text = await inference(audio_data)
    print(text)


In [ ]:
async def playback(audio_data, samplerate):
# def playback(audio_data, samplerate):
    sd.play(audio_data.astype(float)/4096, samplerate=samplerate)
    # raise Exception("not consumed")

In [ ]:
audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=SAMPLING_RATE, input=True, frames_per_buffer=CHUNK_SIZE)


# stream.stop_stream()
# stream.close()
# audio.terminate()

In [ ]:

while True:
    data = stream.read(int(DETECT_DURATION * SAMPLING_RATE), exception_on_overflow=False)
    audio_data_in = np.frombuffer(data, dtype=np.int16)

    # print(audio_data_in[:20])
    # print("detecting:", np.max(audio_data_in))

    if np.max(audio_data_in) < DETECT_THRESHOLD:
        continue
    
    print(np.max(audio_data_in))

    audio_data = audio_data_in
    while np.max(audio_data_in) > DETECT_THRESHOLD:
        data = stream.read(int(RECORD_DURATION * SAMPLING_RATE), exception_on_overflow=False)
        audio_data_in = np.frombuffer(data, dtype=np.int16)
        audio_data = np.concatenate((audio_data, audio_data_in))
        if(len(audio_data) > SAMPLING_RATE*30):
            break

    # data = stream.read(int(RECORD_DURATION * SAMPLING_RATE))
    # audio_data_record = np.frombuffer(data, dtype=np.int16)

    # audio_data = np.concatenate((audio_data_in, audio_data_record))
    await playback(audio_data, SAMPLING_RATE)
    # audio_data = audio_data_in
    
    print("audio_data min,max = ", np.min(audio_data), np.max(audio_data), audio_data.shape)



    # # Reduce noise from the audio using noisereduce
    # audio_noise_reduced = nr.reduce_noise(y=audio_data, sr=SAMPLING_RATE)
    
    # sd.play(audio_data, samplerate=SAMPLING_RATE)

    # print("audio_noise_reduced min,max = ", np.min(audio_noise_reduced), np.max(audio_noise_reduced), audio_noise_reduced.shape)

    # await stt_func(audio_data.astype(np.float32))
    
    # text = inference(audio_noise_reduced)
    # text = inference(audio_noise_reduced.astype(np.float32))
    # text = inference(audio_data.astype(np.float32))

    # flag_skip = text_filter(text)

    # if flag_skip:
    #     print("skip +: ", text)
    #     continue

    # print(text)
    

    stream.stop_stream()
    stream.close()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=SAMPLING_RATE, input=True, frames_per_buffer=CHUNK_SIZE)


    if not listener.running:
        break

In [ ]:
# # Create a coroutine
# coro = asyncio.sleep(1, result=3)

# loop = asyncio.get_event_loop()
# timeout = 5
# # Submit the coroutine to a given loop
# future = asyncio.run_coroutine_threadsafe(coro, loop)

# # Wait for the result with an optional timeout argument
# assert future.result(timeout) == 3

# try:
#     result = future.result(timeout)
# except TimeoutError:
#     print('The coroutine took too long, cancelling the task...')
#     future.cancel()
# except Exception as exc:
#     print(f'The coroutine raised an exception: {exc!r}')
# else:
#     print(f'The coroutine returned: {result!r}')

In [ ]:
# import os
# import time



In [ ]:

# import subprocess
# subprocess.Popen(["dir"])

In [ ]:
# from multiprocessing import Process, Manager

# def f(d, l):
#     d[1] = '1'
#     d['2'] = 2
#     d[0.25] = None
#     l.reverse()

# if __name__ == '__main__':
#     with Manager() as manager:
#         d = manager.dict()
#         l = manager.list(range(10))

#         p = Process(target=f, args=(d, l))
#         p.start()
#         p.join()

#         print(d)
#         print(l)

In [ ]:
# import asyncio
# import concurrent.futures
# import threading
# import time
# import multiprocessing


In [ ]:


# # loop = asyncio.get_running_loop()

# # IO_bound -> thread pool
# # CPU_bound -> process pool

# # # 3. Run in a custom process pool:
# # # with concurrent.futures.ThreadPoolExecutor() as pool:
# # with concurrent.futures.ProcessPoolExecutor() as pool:
# #     result = await loop.run_in_executor(
# #         pool, playback)
# #     print('custom process pool', result)
# # asyncio.create_task(playback(audio_data, SAMPLING_RATE))
# await playback(audio_data, SAMPLING_RATE)

In [ ]:
# async def playback(audio_data, samplerate):
def playback(audio_data, samplerate):
    sd.play(audio_data.astype(float)/4096, samplerate=samplerate)
    # raise Exception("not consumed")
    
import time
audio_data = np.zeros(1)
audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=SAMPLING_RATE, input=True, frames_per_buffer=CHUNK_SIZE)
for i in range(3):
    data = stream.read(int(RECORD_DURATION * SAMPLING_RATE))
    # data = stream.read(int(DETECT_DURATION * SAMPLING_RATE), exception_on_overflow=False)
    # audio_data_in = np.frombuffer(data, dtype=np.int16)
    audio_data_in = np.frombuffer(data, dtype=np.int16)
    audio_data = np.concatenate((audio_data, audio_data_in))
    
    # asyncio.create_task(playback(audio_data, SAMPLING_RATE))
    # await playback(audio_data, SAMPLING_RATE)
    # threading.Thread(target=playback, args=(audio_data, SAMPLING_RATE), daemon=True).start()
    mp = multiprocessing.Process(target=playback, args=(audio_data, SAMPLING_RATE), daemon=True)
    mp.start()
    mp.join()

    time.sleep(1)

In [ ]:
#play audio_data
# sd.play(audio_data.astype(float)/4096, samplerate=SAMPLING_RATE)

In [ ]:
import gradio as gr
import numpy as np
import torch
from datasets import load_dataset

from transformers import SpeechT5ForTextToSpeech, SpeechT5HifiGan, SpeechT5Processor, pipeline
from transformers import VitsModel, VitsTokenizer

In [ ]:

model_path ="https://huggingface.co/Kamtera/persian-tts-male1-vits/resolve/main/checkpoint_88000.pth"  # Absolute path to the model checkpoint.pth
config_path ="https://huggingface.co/Kamtera/persian-tts-male1-vits/resolve/main/config.json" # Absolute path to the model config.json


In [ ]:
# audio_input_queue = Queue(maxsize=20)
audio_input_queue = Queue()
audio_speech2text_queue = Queue()
stop_record_event = threading.Event()


In [ ]:


# Define your constants (FORMAT, CHANNELS, RATE, CHUNK_SIZE, RECORD_DURATION, DETECT_THRESHOLD)

# def audio_processing():
    # audio = pyaudio.PyAudio()
    # stream = audio.open(format=FORMAT, channels=CHANNELS, rate=SAMPLING_RATE, input=True, frames_per_buffer=CHUNK_SIZE)

    # while True:
    #     data = stream.read(int(DETECT_DURATION * SAMPLING_RATE))
    #     audio_data_in = np.frombuffer(data, dtype=np.int16)

    #     # print(audio_data_in[:20])
    #     # print("detecting:", np.max(audio_data_in))

    #     if np.max(audio_data_in) < DETECT_THRESHOLD:
    #         continue
        
    #     print(np.max(audio_data_in))

    #     audio_data = audio_data_in
        
    #     print("audio_data min,max = ", np.min(audio_data), np.max(audio_data), audio_data.shape)

    #     # Reduce noise from the audio using noisereduce
    #     audio_noise_reduced = nr.reduce_noise(y=audio_data, sr=SAMPLING_RATE)
        
    #     # sd.play(audio_noise_reduced, samplerate=SAMPLING_RATE)

    #     print("audio_noise_reduced min,max = ", np.min(audio_noise_reduced), np.max(audio_noise_reduced), audio_noise_reduced.shape)

    #     # while np.max(audio_data_in) >= DETECT_THRESHOLD:
    #     #     data = stream.read(int(RECORD_DURATION * RATE))
    #     #     audio_data_in = np.frombuffer(data, dtype=np.int16)
    #     #     print("input voice max = ", np.max(audio_data_in), audio_data_in.shape)
    #     #     audio_data = np.concatenate((audio_data, audio_data_in))

    #     # print("audio_data.shape = ", audio_data.shape)
    #     audio_input_queue.put(audio_noise_reduced)
        

# # Create and start the thread as a daemon
# audio_input_thread = threading.Thread(target=audio_processing, daemon=True)
# audio_input_thread.start()

# # You can continue with other tasks here...

# # Optionally, you can wait for the audio thread to finish if needed
# # audio_input_thread.join()


In [ ]:
while True:
    if not audio_input_queue.empty():
        audio_data = np.zeros((1,))
        # for i in range(10):
        audio_input = audio_input_queue.get()
        audio_input = np.concatenate((audio_input, audio_input_queue.get()))
        sd.play(audio_input, samplerate=SAMPLING_RATE)
        # sd.wait()
        audio_input_queue.task_done()

    if not listener.running:
        break

In [ ]:
# def audio_playing():
#     while(True):    
#         audio_data = audio_input_queue.get()
#         # print("C: data = ", data)
#         # data_with_id = f"Task C: {data}"
#         print("receive audio data, len = ", len(audio_data), "type = ", type(audio_data))
#         sd.play(audio_data, SAMPLING_RATE)
#         # sd.wait()
#         audio_input_queue.task_done()

#         if not listener.running:
#             break

# # Create and start the thread as a daemon
# audio_playing_thread = threading.Thread(target=audio_playing, daemon=True)
# audio_playing_thread.start()

In [ ]:
def audio_speech2text():
    while(True):    
        audio_data = audio_input_queue.get()
        print("stt: received audio data = ", len(audio_data))
        # data_with_id = f"Task C: {data}"
        audio_speech2text = inference(audio_data.astype(np.float32))
        # sd.play(audio_data, SAMPLING_RATE)
        # sd.wait()
        audio_speech2text_queue.put(audio_speech2text)
        audio_input_queue.task_done()

        if not listener.running:
            break

# Create and start the thread as a daemon
audio_speech2text_thread = threading.Thread(target=audio_speech2text, daemon=True)
audio_speech2text_thread.start()

In [ ]:
def text_printing():
    while(True):    
        text = audio_speech2text_queue.get()
        # print("C: data = ", data)
        # data_with_id = f"Task C: {data}"
        print("receive text data, len = ", len(text), text)
        audio_speech2text_queue.task_done()

        if not listener.running:
            break

# Create and start the thread as a daemon
text_printing_thread = threading.Thread(target=text_printing, daemon=True)
text_printing_thread.start()

In [ ]:
import sys
# Function to forcefully terminate all threads
def force_terminate_threads():
    sys.exit()
    audio_input_thread.join()
    audio_playing_thread.join()

# Register the force_terminate_threads function to be called at program exit
atexit.register(force_terminate_threads)

In [ ]:
import gradio as gr
import numpy as np
import torch
from datasets import load_dataset

from transformers import SpeechT5ForTextToSpeech, SpeechT5HifiGan, SpeechT5Processor, pipeline
from transformers import VitsModel, VitsTokenizer

# from TTS.config import load_config
# from TTS.utils.manage import ModelManager
# from TTS.utils.synthesizer import Synthesizer
# import torchaudio
# from TTS.api import TTS


# tts=TTS(model_path="https://huggingface.co/Kamtera/persian-tts-male1-vits/resolve/main/checkpoint_88000.pth",
#         config_path="https://huggingface.co/Kamtera/persian-tts-male1-vits/resolve/main/config.json")

model_path ="https://huggingface.co/Kamtera/persian-tts-male1-vits/resolve/main/checkpoint_88000.pth"  # Absolute path to the model checkpoint.pth
config_path ="https://huggingface.co/Kamtera/persian-tts-male1-vits/resolve/main/config.json" # Absolute path to the model config.json

# text_fa=".زندگی فقط یک بار است؛ از آن به خوبی استفاده کن"


# synthesizer = Synthesizer(
#     model_path, config_path
# )

# synthesizer = Synthesizer(
#     model_path, config_path
# )
# wavs = synthesizer.tts(text_fa)


device = "cuda:0" if torch.cuda.is_available() else "cpu"

# load speech translation checkpoint
asr_pipe = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)

# load text-to-speech checkpoint and speaker embeddings
# processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

# model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)
# vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)




# embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
# speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# model = VitsModel.from_pretrained("Matthijs/mms-tts-deu").to(device)
model = VitsModel.from_pretrained("Matthijs/mms-tts-deu")
tokenizer = VitsTokenizer.from_pretrained("Matthijs/mms-tts-deu")


from transformers import MT5ForConditionalGeneration, MT5Tokenizer

# en_fa_trans_model_size = "base"
# en_fa_trans_model_name = f"persiannlp/mt5-{en_fa_trans_model_size}-parsinlu-translation_en_fa"
# en_fa_trans_tokenizer = MT5Tokenizer.from_pretrained(en_fa_trans_model_name)
# en_fa_trans_model = MT5ForConditionalGeneration.from_pretrained(en_fa_trans_model_name)

# def translate_en2fa(input_string, **generator_args):
#     input_ids = en_fa_trans_tokenizer.encode(input_string, return_tensors="pt")
#     res = en_fa_trans_model.generate(input_ids, **generator_args)
#     output = en_fa_trans_tokenizer.batch_decode(res, skip_special_tokens=True)
#     print(input_string, output)
#     return output


# translate_en2fa("Praise be to Allah, the Cherisher and Sustainer of the worlds;")


def translate(audio):
    # outputs = asr_pipe(audio, max_new_tokens=256, generate_kwargs={"task": "translate"})
    # # outputs = asr_pipe(audio, max_new_tokens=256, generate_kwargs={"language":"<|fa|>","task": "transcribe"})
    outputs = asr_pipe(audio, max_new_tokens=256, generate_kwargs={"language":"<|de|>","task": "transcribe"})
    return outputs["text"]


def synthesise(text):
    # inputs = processor(text=text, return_tensors="pt")
    # speech = model.generate_speech(inputs["input_ids"].to(device), speaker_embeddings.to(device), vocoder=vocoder)

    
    inputs = tokenizer(text, return_tensors="pt")
    # inputs = [x.to(device) for x in inputs]
    # input_ids = inputs["input_ids"].to(device)
  
    with torch.no_grad():
        # outputs = model(input_ids)
        output = model(**inputs).waveform

    sample_rate=model.config.sampling_rate
    speech=output.cpu().numpy()
    
    # speech = outputs.audio[0]

    
    # tts.tts_to_file(text,file_path='output_fa.wav')
    # outputs, sample_rate = torchaudio.load("output_fa.wav")
    # speech = outputs[0]

    # wavs = synthesizer.tts(text)
    # speech = torch.tensor(wavs)[None]
    # sample_rate = 22050

    
    return speech, sample_rate


def speech_to_speech_translation(audio):
    translated_text = translate(audio)
    # print(translated_text)
    # translated_text = translate_en2fa(translated_text)
    # translated_text = str(translated_text)
    # print(translated_text)
    synthesised_speech, sample_rate = synthesise(translated_text)
    # print(synthesised_speech.shape, sample_rate)
    # print(synthesised_speech.max(), synthesised_speech.min())
    # synthesised_speech = (synthesised_speech.numpy() * 32767).astype(np.int16)
    synthesised_speech = ( (synthesised_speech - synthesised_speech.min()) * 10000).astype(np.int16)[0]
      
    # print(synthesised_speech.shape, sample_rate)
    # print(synthesised_speech.max(), synthesised_speech.min())

    # import sounddevice as sd
    # sd.play(synthesised_speech, samplerate=sample_rate)
    # sd.wait()

    return sample_rate, synthesised_speech


title = "Cascaded STST"
description = """
Demo for cascaded speech-to-speech translation (STST), mapping from source speech in any language to target speech in English. Demo uses OpenAI's [Whisper Base](https://huggingface.co/openai/whisper-base) model for speech translation, and Microsoft's
[SpeechT5 TTS](https://huggingface.co/microsoft/speecht5_tts) model for text-to-speech:
![Cascaded STST](https://huggingface.co/datasets/huggingface-course/audio-course-images/resolve/main/s2st_cascaded.png "Diagram of cascaded speech to speech translation")
"""

demo = gr.Blocks()

mic_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy"),
    title=title,
    description=description,
)

file_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(source="upload", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy"),
    # examples=[["./example.wav"]],
    title=title,
    description=description,
)

with demo:
    gr.TabbedInterface([mic_translate, file_translate], ["Microphone", "Audio File"])

demo.launch()